<a href="https://colab.research.google.com/github/NonKhuna/word-correction/blob/main/DeepGI_clean_correction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install library
- Download and import library 
- Download necessaryfile 
  - Keywords file
  - Data which are the result of ASR

In [ ]:
# install library
! pip --q install pythainlp
! pip --q install fuzzywuzzy[speedup]
! pip --q install evaluate
! pip --q install jiwer
! pip --q install marisa_trie
! pip --q install simpletransformers==0.63.9 #0.63.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# utilize
import re
import time
import json
import pandas as pd
from tqdm import tqdm

# evaluate result
from fuzzywuzzy import fuzz
from evaluate import load

# process data
import marisa_trie
from collections import defaultdict
from pythainlp import word_tokenize
from pythainlp.util import normalize 

# **close warning of updating transformers
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# split data
from sklearn.model_selection import train_test_split

# Load necessary files

In [ ]:
!gdown 1QT4RkeXe8KjS6TbzQVvbVAxnp1FOXWrF # Load file keywords 
!gdown 1JFoQSW1AQ3f9hWqUhSC2mCps2rT-iTcy # Load data (the result from ASR)

Downloading...
From: https://drive.google.com/uc?id=1QT4RkeXe8KjS6TbzQVvbVAxnp1FOXWrF
To: /content/keywords_deepGI.json
100% 1.23k/1.23k [00:00<00:00, 6.86MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JFoQSW1AQ3f9hWqUhSC2mCps2rT-iTcy
To: /content/model2_test.csv
100% 700k/700k [00:00<00:00, 161MB/s]


# Data preparation
- Load data into a dataframe.

In [ ]:
# read file csv
df = pd.read_csv("/content/model2_test.csv", names=['target_text', 'input_text'], header=None)
df.shape

(2688, 2)

In [ ]:
# split dataset
X = df[['input_text']]
y = df[['target_text']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train = X_train.join(y_train)
test = X_test.join(y_test)
train.head()

,input_text,target_text
610,Hello DeepGI เล็กกว่า 5 มิลลิเมตร Sessile Canc...,Hello DeepGI เล็กกว่า 5 มิลลิเมตร Sessile Canc...
2004,Hello DeepGI เล็กกว่า 5 มิลลิเมตร Sessile Subu...,Hello DeepGI เล็กกว่า 5 มิลลิเมตร Sessile Subm...
1406,Hello DeepGI 5.5 มิลลิเมตร Sessile Lipoma Type...,Hello DeepGI 10.5 มิลลิเมตร Sessile Lipoma Typ...
1213,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Sessile Sub...,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Sessile Sub...
57,Hello DeepGI 5.5 มิลลิเมตร Sessile Lipoma Tide...,Hello DeepGI 10.5 มิลลิเมตร Sessile Lipoma Typ...


# Custom dictionary
- Create a new dictionary using the DeepGI's keywords and target_text (lower_case).
- It is necessary to set a `threshold` value which determines the base frequency for new keys.

In [ ]:
# initiate a dictionary
custom_dict = defaultdict(int)
threshold = 50

# get keywords from the keyword json file. If the token doesn't exist in the dictionary, the dictionary creates a new key with a threshold value.
with open('/content/keywords_deepGI.json') as f :
  data = json.load(f)
  for field in data :
    for token in data[field] :
        custom_dict[token] = threshold

# get from target_text
for i in range(len(df)) :
  for x in word_tokenize(df['target_text'][i].lower()) :
    custom_dict[x] += 1

trie = marisa_trie.Trie(custom_dict.keys())
vocabs = custom_dict.keys()
custom_dict = list(custom_dict.items())
custom_dict[:5]

[('hello', 2738),
 ('deepgi', 2738),
 ('terminal', 50),
 ('ileum', 50),
 ('cecum', 50)]

In [ ]:
# Interesting error
ids = [548, 623, 450, 239]
df.iloc[ids]

,target_text,input_text
548,Hello DeepGI 1 เซนติเมตร Flat Subepithelial Ty...,Hello DeepGI 1 เซนติเมตร Flat Submitting ฟัตร ...
623,Hello DeepGI เล็กกว่า 5 มิลลิเมตร Sessile Canc...,Hello DeepGI րեկกว่า 5 มิลลิเมตร Sessile Canc...
450,Hello DeepGI 1 เซนติเมตร Pedunculated Solitary...,Hello DeepGI 1 เซนติเมตร Pedaculated Solitary ...
239,Hello DeepGI 10.5 มิลลิเมตร Flat Hemorrhoid Ty...,Hello DeepGI 5.5 มิลลิเมตร Fat Hemorrhoid ถ่าย...


# First evaluate
For comparison.
- create a evaluate `evaluate_df` function.

In [ ]:
import re
def get_nums(text) :
  regx = r'\b\d+(?:\.\d+)?\b|\.\d+\b'
  numbers = []
  for match in re.findall(regx, text):
    numbers.append(match)

  return numbers

def evaluate_df(df, target_name="target_text", input_name="input_text", lower_case=True):
    """
    Evaluates a DataFrame to check for missing data, error words, and wrong words.
    Args:
        df (pandas.DataFrame): The DataFrame to evaluate.
        target_name (str): The name of the column containing the target text. Defaults to "target_text".
        input_name (str): The name of the column containing the input text. Defaults to "input_text".
        lower_case (boolean): if true, The model compare in lower case.
    Returns:
        None
    """
    def wrong_word(data):
        """Checks if there are any words in the input text that are not present in the target text.

        Args:
            data (pandas.Series): A row of data from the DataFrame.

        Returns:
            bool: True if there is at least one word in the input text that is not present in the target text, False otherwise.
        """
        if word_error(data): return False
        target = set(word_tokenize(data[target_name]))
        for x in word_tokenize(data[input_name]):
          if x not in target:
            return True
        return False

    def word_error(data):
        """Checks if there are any words in the input text that are not present in a trie data structure and do not match a regular expression for numbers.

        Args:
            data (pandas.Series): A row of data from the DataFrame.

        Returns:
            bool: True if there is at least one word that meets these conditions, False otherwise.
        """
        for x in word_tokenize(data[input_name]):
          if not x in trie and not re.match(f"^[\d\.]+$", x):
            return True
        return False

    def missing_data(data):
        """Compares the number of words in the target text to the number of words in the input text.

        Args:
            data (pandas.Series): A row of data from the DataFrame.

        Returns:
            str: "missing" if there are more words in the target text than in the input text, "exceed" if there are more words in the input text than in the target text, and "same" if there are an equal number of words in both.
        """
        target = len(word_tokenize(data[target_name]))
        input = len(word_tokenize(data[input_name]))
        if target > input:
            return "missing"
        elif target < input:
            return "exceed"
        else:
            return "same"
    
    def wrong_number(data) :
        c_r = get_nums(data[target_name])
        c_i = get_nums(data[input_name])
        if c_r != [] and c_i != c_r :
          return True
        return False
    
    df_temp = df.copy()
    df_temp[input_name] = df_temp[input_name].str.lower().str.strip()
    df_temp[target_name] = df_temp[target_name].str.lower().str.strip()
    
    df_temp['word_error'] = df_temp.apply(word_error, axis=1)
    df_temp['missing_data'] = df_temp.apply(missing_data, axis=1)
    df_temp['wrong_word'] = df_temp.apply(wrong_word, axis=1)
    df_temp['wrong_number'] = df_temp.apply(wrong_number, axis=1)

    word_error_rate = sum(df_temp['word_error'])/ len(df_temp)
    exceed_data_rate = sum(df_temp['missing_data']=="exceed") / len(df_temp)
    missing_data_rate = sum(df_temp['missing_data']=="missing") / len(df_temp)
    wrong_word_rate = sum(df_temp['wrong_word']) / len(df_temp)
    wrong_number_rate = sum(df_temp['wrong_number']) / len(df_temp)
  
    wer = load("wer")
    wer_score = wer.compute(predictions=df_temp[input_name], references=df_temp[target_name])

  

    df_temp[input_name+"_isSame"] = df_temp.apply(lambda x: x[target_name] == x[input_name], axis=1)
    print(f"\
    word_error_rate: {word_error_rate}\n\
    wrong_number_rate: {wrong_number_rate}\n\
    exceed_data_rate: {exceed_data_rate}\n\
    missing_data_rate: {missing_data_rate}\n\
    wrong_word_rate: {wrong_word_rate}\n\
    wer_score: {wer_score}\n\
    accuracy: {sum(df_temp[input_name+'_isSame']) / df_temp.shape[0]}\
    ")
    
    if lower_case  :
      df_temp[input_name] = df[input_name]
      df_temp[target_name] = df[target_name]
    return df_temp

In [ ]:
df = evaluate_df(df, lower_case=True)

    word_error_rate: 0.4799107142857143
    wrong_number_rate: 0.20461309523809523
    exceed_data_rate: 0.08035714285714286
    missing_data_rate: 0.06919642857142858
    wrong_word_rate: 0.06398809523809523
    wer_score: 0.11743273542600897
    accuracy: 0.45163690476190477    


In [ ]:
df

,target_text,input_text,word_error,missing_data,wrong_word,wrong_number,input_text_isSame
0,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,False,same,False,False,True
1,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,False,same,False,False,True
2,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,Hello DeepGI 5.5 มิลลิเมตร Sessile Polyp NBI B...,False,same,True,True,False
3,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,Hello DeepGI 5.5 มิลลิเมตร Sessile Polyp NBI B...,False,same,True,True,False
4,Hello DeepGI 10.5 มิลลิเมตร Sessile Polyp NBI ...,Hello DeepGI 5.5 มิลลิเมตร Sessile Polyp NBI R...,False,same,True,True,False
...,...,...,...,...,...,...,...
2683,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Flat Circum...,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Fat Circumf...,True,same,False,False,False
2684,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Flat Circum...,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Fat Circumf...,True,missing,False,True,False
2685,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Flat Circum...,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Flat Circum...,True,same,False,False,False
2686,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Flat Circum...,Hello DeepGI ใหญ่กว่า 10 มิลลิเมตร Fat Circumf...,True,same,False,False,False


# Norvig's approch
Correct the word which has 1-2 distance errors.

## Custom Norvig


In [ ]:
from collections import Counter
from string import digits
from typing import Callable, List, Set, Tuple

from pythainlp import thai_digits, thai_letters
from pythainlp.corpus import tnc
from pythainlp.util import isthaichar

eng_letters = []
# a-z
for ch in range(65, 91):
  eng_letters.append(chr(ch))
# A-Z
for ch in range(97, 123) :
  eng_letters.append(chr(ch))


def _no_filter(word: str) -> bool:
    return True

def _is_thai_and_not_num(word: str) -> bool:
    for ch in word:
        if ch != "." and not isthaichar(ch):
            return False
        if ch in digits or ch in thai_digits:
            return False
    return True

def _keep(
    word_freq: int,
    min_freq: int,
    min_len: int,
    max_len: int,
    dict_filter: Callable[[str], bool],
) -> Callable[[str], bool]:
    """
    Keep only Thai words with at least min_freq frequency
    and has length between min_len and max_len characters
    """
    if not word_freq or word_freq[1] < min_freq:
        return False

    word = word_freq[0]
    if not word or len(word) < min_len or len(word) > max_len or word[0] == ".":
        return False

    return dict_filter(word)


def _edits1(word: str) -> Set[str]:
    """
    Return a set of words with edit distance of 1 from the input word
    """
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in eng_letters]
    inserts = [L + c + R for L, R in splits for c in eng_letters]

    return set(deletes + transposes + replaces + inserts)


def _edits2(word: str) -> Set[str]:
    """
    Return a set of words with edit distance of 2 from the input word
    """
    return set(e2 for e1 in _edits1(word) for e2 in _edits1(e1))


class CustomNorvigSpellChecker:
      def __init__(
          self,
          custom_dict: List[Tuple[str, int]] = None,
          min_freq: int = 2,
          min_len: int = 2,
          max_len: int = 20,
          dict_filter: Callable[[str], bool] = _is_thai_and_not_num,
      ):
        """
        Initialize Peter Norvig's spell checker object

        :param str custom_dict: A list of tuple (word, frequency) to create a spelling dictionary. Default is from Thai National Corpus (around 40,000 words).
        :param int min_freq: Minimum frequency of a word to keep (default = 2)
        :param int min_len: Minimum length (in characters) of a word to keep (default = 2)
        :param int max_len: Maximum length (in characters) of a word to keep (default = 40)
        :param func dict_filter: A function to filter the dictionary. Default filter removes any word with number or non-Thai characters. If no filter is required, use None.
        """
        if not custom_dict:  # default, use Thai National Corpus
            custom_dict = tnc.word_freqs()

        if not dict_filter:
            dict_filter = _no_filter

        # filter word list
        custom_dict = [
            word_freq
            for word_freq in custom_dict
            if _keep(word_freq, min_freq, min_len, max_len, dict_filter)
        ]

        self.__WORDS = Counter(dict(custom_dict))
        self.__WORDS_TOTAL = sum(self.__WORDS.values())
        if self.__WORDS_TOTAL < 1:
            self.__WORDS_TOTAL = 0
        
        self.max_len = max_len

      def dictionary(self) -> List[Tuple[str, int]]:
          """
          Return the spelling dictionary currently used by this spell checker
          """
          return self.__WORDS.items()


      def known(self, words: List[str]) -> List[str]:
          """
          Return a list of given words that found in the spelling dictionary

          :param str words: A list of words to check if they are in the spelling dictionary
          """
          return list(w for w in words if w in self.__WORDS)


      def prob(self, word: str) -> float:
          """
          Return probability of an input word, according to the spelling dictionary

          :param str word: A word to check its probability of occurrence
          """
          return self.__WORDS[word] / self.__WORDS_TOTAL


      def freq(self, word: str) -> int:
          """
          Return frequency of an input word, according to the spelling dictionary

          :param str word: A word to check its frequency
          """
          return self.__WORDS[word]


      def spell(self, word: str) -> List[str]:
          """
          Return a list of possible words, according to edit distance of 1 and 2,
          sorted by frequency of word occurrance in the spelling dictionary

          :param str word: A word to check its spelling
          """
          if len(word) > self.max_len :
            return word

          if not word:
              return ""

          candidates = (
              self.known([word])
              or self.known(_edits1(word))
              or self.known(_edits2(word))
              or [word]
          )
          candidates.sort(key=self.freq, reverse=True)

          return candidates


      def correct(self, word: str) -> str:
          """
          Return the most possible word, using the probability from the spelling dictionary
          :param str word: A word to correct its spelling
          """
          if not word:
              return ""

          return self.spell(word)[0]

## `Class DeepGICorrector()`

In [ ]:
class DeepGICorrector():
  def __init__(self, custom_dict, max_sentence_length=70, lower_case=True) :
    self.corrector = CustomNorvigSpellChecker(custom_dict, dict_filter=self.fillter_num)
    self.max_sentence_length = max_sentence_length
    self.lower_case = lower_case
  
  def fillter_num(self, word) :
    for ch in word:
      if ch in digits or ch in thai_digits:
        return False
    return True

  def correct(self, words) :
    if self.lower_case :
      words = words.lower()
    words = normalize(words)
    tokens = word_tokenize(words)
    results = []
    for i,token in enumerate(tokens) :
      if i > self.max_sentence_length :
        results.append(token)
        continue
      if self.contain_num(token) or (token in trie) or token == " ":
        results.append(token)
      else :
        token_r = self.corrector.correct(token)
        results.append(token_r)
    results = "".join(results)
    return results
  
  def contain_num(self, word) :
    for ch in word:
      if ch in digits or ch in thai_digits:
        return True
    return False

## Demo

In [ ]:
sentence = df['input_text'][42] + "บริเวณ anus polpb"
corrector = DeepGICorrector(custom_dict, lower_case=True)

start = time.time()

print("Before:", sentence)
print("After:",corrector.correct(sentence))
end = time.time()

print("Time:",end - start)

Before: Hello DeepGI 10.5 มิลลิเมตร Sessile Cancer Thai 3 Pseudopolyp Anus EMR บริเวณ anus polpb
After: hello deepgi 10.5 มิลลิเมตร sessile cancer thai 3 pseudopolyp anus emr บริเวณ anus polyp
Time: 0.9365456104278564


## Perform correction

In [ ]:
corrector = DeepGICorrector(custom_dict)
corrected_texts = []

for i in tqdm(range(len(df))) :
  corrected_texts.append(corrector.correct(df['input_text'][i]))
df['clean_1_input'] = corrected_texts

100%|██████████| 2688/2688 [10:39<00:00,  4.20it/s]


## Evaluation

In [ ]:
df = evaluate_df(df, "target_text", "clean_1_input", lower_case=True)

    word_error_rate: 0.3005952380952381
    wrong_number_rate: 0.20461309523809523
    exceed_data_rate: 0.07961309523809523
    missing_data_rate: 0.06994047619047619
    wrong_word_rate: 0.10416666666666667
    wer_score: 0.09786995515695067
    accuracy: 0.5863095238095238    


# Model 1
Pre-trained BART with Simple-transformers

## Prepare dataset (train/test)

In [ ]:
# split dataset
X = df[['input_text']]
y = df[['target_text']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train = X_train.join(y_train)
test = X_test.join(y_test)
train.head()

## `Class DeepGIBartCorrection()`

In [ ]:
import pandas as pd
import torch
import logging
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)



class DeepGIBartCorrection() :
  def __init__(self, epochs=3, max_length=60, batch_size=4, model_name="facebook/m2m100_418M", src_lang="th_TH", tgt_leng="th_TH", lower_case=True, mask_num=True):
    
    logging.basicConfig(level=logging.INFO)
    transformers_logger = logging.getLogger("transformers")
    transformers_logger.setLevel(logging.WARNING)

    # set arguments
    model_args = Seq2SeqArgs()
    model_args.num_train_epochs = epochs
    model_args.no_save = True
    model_args.evaluate_generated_text = True
    model_args.evaluate_during_training = True
    model_args.eval_batch_size = batch_size
    model_args.train_batch_size= batch_size
    model_args.overwrite_output_dir = True
    model_args.max_length = max_length
    model_args.length_penalty = 2

    # Initialize model
    self.model = Seq2SeqModel(
        encoder_decoder_type="mbart",
        encoder_decoder_name=model_name,
        args=model_args,
        use_cuda= torch.cuda.is_available(),
        src_lang=src_lang, tgt_lang=tgt_leng
    )

    self.lower_case = lower_case
    self.mask_num = mask_num

  def count_matches(self, labels, preds):
      return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
  
  def fit(self, train_set, validation_set):
      train_pre = train_set
      eval_pre = validation_set

      train_pre['input_text'] = train_set['input_text'].map(lambda x: self.preprocess_text(x)[0])
      train_pre['target_text'] = train_set['target_text'].map(lambda x: self.preprocess_text(x)[0])
      eval_pre['input_text'] = eval_pre['input_text'].map(lambda x: self.preprocess_text(x)[0])
      eval_pre['target_text'] = eval_pre['target_text'].map(lambda x: self.preprocess_text(x)[0])

      if self.lower_case :
        train_pre['input_text'] = train_pre['input_text'].str.lower()
        train_pre['target_text'] = train_pre['target_text'].str.lower()
        eval_pre['input_text'] = eval_pre['input_text'].str.lower()
        eval_pre['target_text'] = eval_pre['target_text'].str.lower()

      # Train the model
      self.model.train_model(train_pre, eval_data=eval_pre, matches=self.count_matches)
      return None

  def evaluate(self, data):
      # Evaluate the model
      data_copy = data
      if self.lower_case :
        data_copy['input_text'] = data_copy['input_text'].str.lower()
        data_copy['target_text'] = data_copy['target_text'].str.lower()
      return self.model.eval_model(data_copy)
  
  def preprocess_text(self, input) :
      """Preprocesses the input text by replacing all numbers with a special token `#`

        Args: 
          input (Union[str, List[str]]): The input text to preprocess

        Returns:
          Union[Tuple[str, List[str]], Tuple[List[str], List[List[str]]]]:
            If the input is a single string, return a tuple containing the preprocessed text 
              and a list of the original numbers
            If the input is a list of strings, return a tuple containing a list of preprocessed texts
            and a list of lists of the original numbers.

      """
      if not self.mask_num :
        return input, []
      regx = r'\b\d+(?:\.\d+)?\b|\.\d+\b'
      if type(input) == str :
        text = input
        numbers = []
        for match in re.findall(regx, text):
          numbers.append(match)

        # Replace all numbers with a special token
        text = re.sub(regx, '#', text)
        return text, numbers
      elif type(input) == list :
        list_numbers = []
        new_texts =  []
        for text in input :
          numbers = []
          for match in re.findall(regx, text):
            numbers.append(match)

          # Replace all numbers with a special token
          text = re.sub(regx, '#', text)
          list_numbers.append(numbers)
          new_texts.append(text)
        return new_texts, list_numbers

  def postprocess_text(self, input_text, input_numbers):
      """Postprocesses the input text by inserting the original numbers back into the output text at the same positions.

        Args:
            input_text (Union[str, List[str]]): The input text to postprocess. Can be a single string or a list of strings.
            input_numbers (Union[List[str], List[List[str]]]): The original numbers to insert back into the output text. Can be a single list of strings or a list of lists of strings.

        Returns:
            Union[str, List[str]]: If the input is a single string, returns the postprocessed text. If the input is a list of strings, returns a list of postprocessed texts.
      """
      if not self.mask_num :
        return input_text

      if type(input_text) == str :
        text = input_text 
        numbers = input_numbers
        it = 0
        stack = 0
        for match in re.finditer(r'#', text):
            if (it == len(numbers)) :
              return "false postprocess"
            text = text[:match.start()+stack] + numbers[it] +  text[match.end()+stack:]
            stack += len(numbers[it]) - 1
            it+=1
        return text
      elif type(input_text) == list :
        out_text = []
        for i, text in enumerate(input_text) :
          numbers = input_numbers[i]
          it = 0
          stack = 0
          for match in re.finditer(r'#', text):
              if (it == len(numbers)) :
                continue
              text = text[:match.start()+stack] + numbers[it] +  text[match.end()+stack:]
              stack += len(numbers[it]) - 1
              it+=1
          out_text.append(text)
        return out_text

  def predict(self, data):

      data_c = data
      if self.lower_case :
        if type(data_c) == str :
          data_c = data_c.lower()
        else :
          data_c = [x.lower() for x in data_c]

      texts, nums = self.preprocess_text(data_c)
      results = self.model.predict(texts)
      results = self.postprocess_text(results, nums)

      # remove ro_RO
      if type(results) == str:
        results = re.sub('ro_RO', "", results).strip()
      elif type(results) == list:
        results = [re.sub('ro_RO', "", x).strip() for x in results]
      return results

## Demo

In [ ]:
# initial model
corrector = DeepGIBartCorrection(epochs=3, max_length=60, batch_size=8, mask_num=True)

# Train model
corrector.fit(train, test)


# model prediction
samples = [
        "hello deepgi 10.5 มิลลิเมตร sessile cancer ถ่าย 3 มิลลิเมตร pseudopolyp anus biosp".lower(),
        "hello deepgi 10.5 มิลลิเมตร sessile cancer thai 3 มิลลิเมตร pseudopolyp anus biosssp".lower(),
         "Hello DeepGI Rectu Solitary rectal ulcer synซัome เล".lower(),
         "Hello DeepGI Rectu Soldffitary rectal ulcer syndrome ".lower(),
         'hello deepgi 5.5 มิลลิเมตร sessile polyp ถ่าย 3 เสือ pseudopolyp anus biospy'.lower(),
         'hello deepgi 1 เซนติเมตร flat submitting ฟัตร submitting ฮิลลิว nbi hyperplastic polyp ascending esd'.lower(),
         'Rectu solitary'.lower(),
         'pour about preparation with solid stone throughout the right column'.lower(),
         'hot snare polypectomy'.lower(),
         'flat polp 3 มิลลิเมตร ทิตรันสเมตร'.lower()
        ]

predicts = corrector.predict(samples)
for i,s in enumerate(samples) :
  print(f"input: {s} \nresult: {predicts[i]}\n")

You are using a model of type m2m_100 to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.
Some weights of MBartForConditionalGeneration were not initialized from the model checkpoint at facebook/m2m100_418M and are newly initialized: ['model.decoder.layernorm_embedding.bias', 'model.encoder.embed_positions.weight', 'model.encoder.layernorm_embedding.weight', 'model.decoder.embed_positions.weight', 'model.encoder.layernorm_embedding.bias', 'model.decoder.layernorm_embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'M2M100Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary,

  0%|          | 0/2150 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/269 [00:00<?, ?it/s]

  0%|          | 0/538 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/68 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

input: hello deepgi 10.5 มิลลิเมตร sessile cancer ถ่าย 3 มิลลิเมตร pseudopolyp anus biosp 
result: hello deepgi 10.5 มิลลิเมตร sessile cancer type 3 pseudopolyp anus biopsy

input: hello deepgi 10.5 มิลลิเมตร sessile cancer thai 3 มิลลิเมตร pseudopolyp anus biosssp 
result: hello deepgi 10.5 มิลลิเมตร sessile cancer type 3 pseudopolyp anus biopsy

input: hello deepgi rectu solitary rectal ulcer synซัome เล 
result: hello deepgi เล็กกว่า # มิลลิเมตร flat circumferential solitary rectal ulcer syndrome type # lipoma rectum clipping

input: hello deepgi rectu soldffitary rectal ulcer syndrome  
result: hello deepgi # มิลลิเมตร flat circumferential solitary rectal ulcer syndrome type # solitary rectal ulcer syndrome rectum clipping

input: hello deepgi 5.5 มิลลิเมตร sessile polyp ถ่าย 3 เสือ pseudopolyp anus biospy 
result: hello deepgi 5.5 มิลลิเมตร sessile polyp type 3 pseudopolyp anus biopsy

input: hello deepgi 1 เซนติเมตร flat submitting ฟัตร submitting ฮิลลิว nbi hyperplastic polyp as

In [ ]:
# Test number encode and decode
d,nums = corrector.preprocess_text("The 456 is a time d-2 and .265, 035.25")
print("example")
print("After encode:", d)
print("After Decode:",corrector.postprocess_text(d, nums))

example
After encode: The # is a time d-# and #, #
After Decode: The 456 is a time d-2 and .265, 035.25


## Evaluate

In [ ]:
df['correct_lm'] = corrector.predict(list(df['input_text']))

Generating outputs:   0%|          | 0/336 [00:00<?, ?it/s]

In [ ]:
df = evaluate_df(df, "target_text", "correct_lm")

    word_error_rate: 0.07738095238095238
    wrong_number_rate: 0.20461309523809523
    exceed_data_rate: 0.024553571428571428
    missing_data_rate: 0.004464285714285714
    wrong_word_rate: 0.13467261904761904
    wer_score: 0.027466367713004484
    accuracy: 0.7872023809523809    


# Model 2 Bert Model (*not used)

In [ ]:
import logging

import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = [
    ["one", "1"],
    ["two", "2"],
]

train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])

eval_data = [
    ["three", "3"],
    ["four", "4"],
]

eval_df = pd.DataFrame(eval_data, columns=["input_text", "target_text"])

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 10,
    "train_batch_size": 2,
    "num_train_epochs": 10,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 15,
    "manual_seed": 4,
}

encoder_type = "roberta"

model = Seq2SeqModel(
    encoder_type,
    "roberta-base",
    "bert-base-cased",
    args=model_args,
    use_cuda=True,
)

model.train_model(train_df)

results = model.eval_model(eval_df)

print(model.predict(["five"]))


model1 = Seq2SeqModel(
    encoder_type,
    encoder_decoder_name="outputs",
    args=model_args,
    use_cuda=True,
)
print(model1.predict(["five"]))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another a

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[',,,,,,, and and,,, and and']


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[',,,,,,, and and,,, and and']
